In [2]:
import torch
import matchzoo as mz

ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.MeanAveragePrecision()
]

In [3]:
train_pack = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
valid_pack = mz.datasets.wiki_qa.load_data('dev', task=ranking_task)

7094272/7094233 [==============================] - 3s 0us/step


In [7]:
preprocessor = mz.models.ArcI.get_default_preprocessor()
train_processed = preprocessor.fit_transform(train_pack)
valid_processed = preprocessor.transform(valid_pack)




Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval:   0%|      | 0/2118 [00:00<?, ?it/s]


Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval:  42%|▍| 880/2118 [00:00<00:00, 8736.00it/s]


Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|█| 2118/2118 [00:00<00:00, 8997.64it/s]



Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval:   0%|    | 0/18841 [00:00<?, ?it/s]


Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval:   3%| | 627/18841 [00:00<00:02, 6223.18it/s]


Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval:   6%| | 1136/18841 [00:00<00:03, 5819.49it/s]


Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval:   9%| | 1657/18841 [00:00<00:03, 5608.93it/s]


Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval:  12%| | 2192/1

TypeError: unhashable type: 'list'

In [ ]:
trainset = mz.dataloader.Dataset(
    data_pack=train_processed,
    mode='pair',
    num_dup=1,
    num_neg=4
)
validset = mz.dataloader.Dataset(
    data_pack=valid_processed,
    mode='point'
)

In [ ]:
padding_callback = mz.models.ArcI.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=32,
    stage='train',
    callback=padding_callback
)
validloader = mz.dataloader.DataLoader(
    dataset=validset,
    batch_size=32,
    stage='dev',
    callback=padding_callback
)

In [ ]:
model = mz.models.ArcI()
model.params['task'] = ranking_task
model.guess_and_fill_missing_params()
model.build()

In [ ]:
optimizer = torch.optim.Adam(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=validloader,
    epochs=10
)

trainer.run()